# Mini Project: Transfer Learning with Keras

Transfer learning is a machine learning technique where a model trained on one task is used as a starting point to solve a different but related task. Instead of training a model from scratch, transfer learning leverages the knowledge learned from the source task and applies it to the target task. This approach is especially useful when the target task has limited data or computational resources.

In transfer learning, the pre-trained model, also known as the "base model" or "source model," is typically trained on a large dataset and a more general problem (e.g., image classification on ImageNet, a vast dataset with millions of labeled images). The knowledge learned by the base model in the form of feature representations and weights captures common patterns and features in the data.

To perform transfer learning, the following steps are commonly followed:

1. Pre-training: The base model is trained on a source task using a large dataset, which can take a considerable amount of time and computational resources.

2. Feature Extraction: After pre-training, the base model is used as a feature extractor. The last few layers (classifier layers) of the model are discarded, and the remaining layers (feature extraction layers) are retained. These layers serve as feature extractors, producing meaningful representations of the data.

3. Fine-tuning: The feature extraction layers and sometimes some of the earlier layers are connected to a new set of layers, often called the "classifier layers" or "task-specific layers." These layers are randomly initialized, and the model is trained on the target task with a smaller dataset. The weights of the base model can be frozen during fine-tuning, or they can be allowed to be updated with a lower learning rate to fine-tune the model for the target task.

Transfer learning has several benefits:

1. Reduced training time and resource requirements: Since the base model has already learned generic features, transfer learning can save time and resources compared to training a model from scratch.

2. Improved generalization: Transfer learning helps the model generalize better to the target task, especially when the target dataset is small and dissimilar from the source dataset.

3. Better performance: By starting from a model that is already trained on a large dataset, transfer learning can lead to better performance on the target task, especially in scenarios with limited data.

4. Effective feature extraction: The feature extraction layers of the pre-trained model can serve as powerful feature extractors for different tasks, even when the task domains differ.

Transfer learning is commonly used in various domains, including computer vision, natural language processing (NLP), and speech recognition, where pre-trained models are fine-tuned for specific applications like object detection, sentiment analysis, or speech-to-text.

In this mini-project you will perform fine-tuning using Keras with a pre-trained VGG16 model on the CIFAR-10 dataset.

First, import all the libraries you'll need.

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard

The CIFAR-10 dataset is a widely used benchmark dataset in the field of computer vision and machine learning. It stands for the "Canadian Institute for Advanced Research 10" dataset. CIFAR-10 was created by researchers at the CIFAR institute and was originally introduced as part of the Neural Information Processing Systems (NIPS) 2009 competition.

The dataset consists of 60,000 color images, each of size 32x32 pixels, belonging to ten different classes. Each class contains 6,000 images. The ten classes in CIFAR-10 are:

1. Airplane
2. Automobile
3. Bird
4. Cat
5. Deer
6. Dog
7. Frog
8. Horse
9. Ship
10. Truck

The images are evenly distributed across the classes, making CIFAR-10 a balanced dataset. The dataset is divided into two sets: a training set and a test set. The training set contains 50,000 images, while the test set contains the remaining 10,000 images.

CIFAR-10 is often used for tasks such as image classification, object recognition, and transfer learning experiments. The relatively small size of the images and the variety of classes make it a challenging dataset for training machine learning models, especially deep neural networks. It also serves as a good dataset for teaching and learning purposes due to its manageable size and straightforward class labels.

Here are your tasks:

1. Load the CIFAR-10 dataset after referencing the documentation [here](https://keras.io/api/datasets/cifar10/).
2. Normalize the pixel values so they're all in the range [0, 1].
3. Apply One Hot Encoding to the train and test labels using the [to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) function.
4. Further split the the training data into training and validation sets using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Use only 10% of the data for validation.  

In [19]:
# Load the CIFAR-10 dataset

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [20]:
print(type(X_train))
print(len(X_train))
print(type(X_test))
print(len(X_test))

<class 'numpy.ndarray'>
50000
<class 'numpy.ndarray'>
10000


In [21]:
# Normalize the pixel values to [0, 1]

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [22]:
# One-hot encode the labels

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [23]:
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


In [24]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [25]:
train_imgDataGen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
    # validation_split=0.1,
    # rescale=1./255
)

val_imgDataGen = ImageDataGenerator()

def custom_generator(generator, X_data, y_data, batch_size, target_size):
    gen = generator.flow(X_data, y_data, batch_size=batch_size)
    while True:
        X_batch, y_batch = next(gen)
        X_batch_resized = np.array([tf.image.resize(image, target_size).numpy() for image in X_batch])
        yield X_batch_resized, y_batch


target_size = (224, 224)
batch_size = 32

train_generator = custom_generator(train_imgDataGen, X_train, y_train, batch_size, target_size)
validation_generator = custom_generator(val_imgDataGen, X_val, y_val, batch_size, target_size)

tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

VGG16 (Visual Geometry Group 16) is a deep convolutional neural network architecture that was developed by the Visual Geometry Group at the University of Oxford. It was proposed by researchers Karen Simonyan and Andrew Zisserman in their paper titled "Very Deep Convolutional Networks for Large-Scale Image Recognition," which was presented at the International Conference on Learning Representations (ICLR) in 2015.

The VGG16 architecture gained significant popularity for its simplicity and effectiveness in image classification tasks. It was one of the pioneering models that demonstrated the power of deeper neural networks for visual recognition tasks.

Key characteristics of the VGG16 architecture:

1. Architecture: VGG16 consists of a total of 16 layers, hence the name "16." These layers are stacked one after another, forming a deep neural network.

2. Convolutional Layers: The main building blocks of VGG16 are the convolutional layers. It primarily uses 3x3 convolutional filters throughout the network, which allows it to capture local features effectively.

3. Max Pooling: After each set of convolutional layers, VGG16 applies max-pooling layers with 2x2 filters and stride 2, which halves the spatial dimensions (width and height) of the feature maps and reduces the number of parameters.

4. Fully Connected Layers: Towards the end of the network, VGG16 has fully connected layers that act as a classifier to make predictions based on the learned features.

5. Activation Function: The network uses the Rectified Linear Unit (ReLU) activation function for all hidden layers, which helps with faster convergence during training.

6. Number of Filters: The number of filters in each convolutional layer is relatively small compared to more recent architectures like ResNet or InceptionNet. However, stacking multiple layers allows VGG16 to learn complex hierarchical features.

7. Output Layer: The output layer consists of 1000 units, corresponding to 1000 ImageNet classes. VGG16 was originally trained on the large-scale ImageNet dataset, which contains millions of images from 1000 different classes.

VGG16 was instrumental in showing that increasing the depth of a neural network can significantly improve its performance on image recognition tasks. However, the main drawback of VGG16 is its high number of parameters, making it computationally expensive and memory-intensive to train. Despite this limitation, VGG16 remains an essential benchmark architecture and has paved the way for even deeper and more efficient models in the field of computer vision, such as ResNet, DenseNet, and EfficientNet.

Here are your tasks:

1. Load [VGG16](https://keras.io/api/applications/vgg/#vgg16-function) as a base model. Make sure to exclude the top layer.
2. Freeze all the layers in the base model. We'll be using these weights as a feature extraction layer to forward to layers that are trainable.

In [26]:
# Load the pre-trained VGG16 model (excluding the top classifier)
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [27]:
# Freeze the layers in the base model

for layer in VGG_model.layers:
  layer.trainable = False

VGG_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

Now, we'll add some trainable layers to the base model.

1. Using the base model, add a [GlobalAveragePooling2D](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) layer, followed by a [Dense](https://keras.io/api/layers/core_layers/dense/) layer of length 256 with ReLU activation. Finally, add a classification layer with 10 units, corresponding to the 10 CIFAR-10 classes, with softmax activation.
2. Create a Keras [Model](https://keras.io/api/models/model/) that takes in approproate inputs and outputs.

In [28]:
# Add a global average pooling layer

x = Flatten()(VGG_model.output)
# x = GlobalAveragePooling2D()(x)

In [29]:
# Add a fully connected layer with 256 units and ReLU activation

x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)


In [30]:
# Add the final classification layer with 10 units (for CIFAR-10 classes) and softmax activation

predictions = Dense(10, activation='softmax')(x)

In [31]:
# Create the fine-tuned model

model = Model(inputs=VGG_model.input, outputs=predictions)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

With your model complete it's time to train it and assess its performance.

1. Compile your model using an appropriate loss function. Feel free to play around with the optimizer, but a good starting optimizer might be Adam with a learning rate of 0.001.
2. Fit your model on the training data. Use the validation data to print the accuracy for each epoch. Try training for 10 epochs. Note, training can take a few hours so go ahead and grab a cup of coffee.

**Optional**: See if you can implement an [Early Stopping](https://keras.io/api/callbacks/early_stopping/) criteria as a callback function.

In [32]:
# Compile the model
from tensorflow.keras.optimizers import legacy

# optimizer = Adam(learning_rate=0.0001)
optimizer = legacy.SGD(learning_rate=0.001, momentum=0.9, decay=1e-4)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

print(steps_per_epoch)
print(validation_steps)

1406
156


In [34]:
# Train the model
# from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
                    train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps=validation_steps,
                    callbacks=[tensorboard_callback],
                    verbose=2
                    )

Epoch 1/20
1406/1406 - 285s - loss: 3.2673 - accuracy: 0.2764 - val_loss: 2.8507 - val_accuracy: 0.4639 - 285s/epoch - 203ms/step
Epoch 2/20
1406/1406 - 270s - loss: 2.9833 - accuracy: 0.3716 - val_loss: 2.7059 - val_accuracy: 0.4770 - 270s/epoch - 192ms/step
Epoch 3/20
1406/1406 - 274s - loss: 2.8202 - accuracy: 0.4193 - val_loss: 2.5892 - val_accuracy: 0.5028 - 274s/epoch - 195ms/step
Epoch 4/20
1406/1406 - 272s - loss: 2.7176 - accuracy: 0.4431 - val_loss: 2.4428 - val_accuracy: 0.5294 - 272s/epoch - 193ms/step
Epoch 5/20
1406/1406 - 286s - loss: 2.6346 - accuracy: 0.4618 - val_loss: 2.3090 - val_accuracy: 0.5675 - 286s/epoch - 204ms/step
Epoch 6/20
1406/1406 - 267s - loss: 2.5641 - accuracy: 0.4753 - val_loss: 2.3070 - val_accuracy: 0.5651 - 267s/epoch - 190ms/step
Epoch 7/20
1406/1406 - 266s - loss: 2.5116 - accuracy: 0.4823 - val_loss: 2.4320 - val_accuracy: 0.5361 - 266s/epoch - 189ms/step
Epoch 8/20
1406/1406 - 260s - loss: 2.4623 - accuracy: 0.4924 - val_loss: 2.1752 - val_acc

With your model trained, it's time to assess how well it performs on the test data.

1. Use your trained model to calculate the accuracy on the test set. Is the model performance better than random?
2. Experiment! See if you can tweak your model to improve performance.  

In [39]:
# Evaluate the model on the test set

# test_loss, test_accuracy = model.evaluate(X_test, y_test)
loss, accuracy = model.evaluate(validation_generator, steps=validation_steps)

print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

156/156 [==============================] - 26s 167ms/step - loss: 1.8204 - accuracy: 0.6486
Test Loss: 1.820394515991211, Test Accuracy: 0.6486378312110901


Model has a 64.86% accuracy score so there is room for improvement, however it is performing better than random chance.

In [40]:
test_generator = custom_generator(val_imgDataGen, X_test, y_test, batch_size, target_size)
test_steps = len(X_test) // batch_size

test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps)

print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

312/312 [==============================] - 53s 171ms/step - loss: 2.1945 - accuracy: 0.5245
Test Loss: 2.1944942474365234, Test Accuracy: 0.5245392918586731
